In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/original-data/data.json


In [2]:
!pip install wikipedia
!pip install rank_bm25
import nltk
nltk.download('punkt')

  Preparing metadata (setup.py) ... - done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11678 sha256=54281e68ac8951684a35379b0e04ba7c0177b830d7e5eab1a0a6a1837cedece8
  Stored in directory: /root/.cache/pip/wheels/5e/b6/c5/93f3dec388ae76edc830cb42901bb0232504dfc0df02fc50de
Successfully built wikipedia
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
import json

# Import from JSON file
with open('/kaggle/input/original-data/data.json', 'r') as json_file:
    data = json.load(json_file)

# Now, imported_data is a list of dictionaries

In [4]:
# import and initialize the tokenizer and model from the checkpoint
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

checkpoint = "sambydlo/bart-large-scientific-lay-summarisation"

tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

tokenizer_config.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.71k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/292 [00:00<?, ?B/s]

In [5]:
content = [page['content'][:10000] for page in data]

In [6]:
from tqdm import tqdm
import nltk

i = 0
final = {}

for flag in tqdm(content):
    sentences = nltk.tokenize.sent_tokenize(flag)
    length = 0
    chunks = []

    for sentence in sentences:
        tokenized_sentence = tokenizer.tokenize(sentence)
        combined_length = len(tokenized_sentence) + length

        if combined_length <= tokenizer.max_len_single_sentence:
            chunks.append(sentence)
            length = combined_length
        else:
            # Handle the case where a sentence itself exceeds the maximum length
            if len(tokenized_sentence) > tokenizer.max_len_single_sentence:
                # Split the sentence into chunks that fit the maximum length
                sub_chunks = [sentence[i:i + tokenizer.max_len_single_sentence]
                              for i in range(0, len(sentence), tokenizer.max_len_single_sentence)]
                chunks.extend(sub_chunks)
            else:
                chunks.append(sentence)
                length = len(tokenized_sentence)




100%|██████████| 100/100 [00:01<00:00, 72.72it/s]


In [7]:
from tqdm import tqdm
import nltk

i = 0
final = {}

for flag in tqdm(content):
    sentences = nltk.tokenize.sent_tokenize(flag)
    length = 0
    chunks = []

    for sentence in sentences:
        tokenized_sentence = tokenizer.tokenize(sentence)
        combined_length = len(tokenized_sentence) + length

        if combined_length <= tokenizer.max_len_single_sentence:
            chunks.append(sentence)
            length = combined_length
        else:
            # Handle the case where a sentence itself exceeds the maximum length
            if len(tokenized_sentence) > tokenizer.max_len_single_sentence:
                # Split the sentence into chunks that fit the maximum length
                sub_chunks = [sentence[i:i + tokenizer.max_len_single_sentence]
                              for i in range(0, len(sentence), tokenizer.max_len_single_sentence)]
                chunks.extend(sub_chunks)
            else:
                chunks.append(sentence)
                length = len(tokenized_sentence)
    inputs = [tokenizer(chunk, return_tensors="pt") for chunk in chunks]
    c_1 = ""
    for input in inputs:
        output = model.generate(**input, max_length = 100)
        x = tokenizer.decode(output[0], skip_special_tokens=True)
        print(x)
        c_1 += x + " "
    print('###########################################################################')
    final[i] = c_1
    i += 1

  0%|          | 0/100 [00:00<?, ?it/s]

Solar energy is radiant light and heat from the Sun that is harnessed using a range of technologies such as solar power to generate electricity, solar thermal energy (including solar water heating ) and solar architecture.
It is an essential source of renewable energy and its technologies are broadly characterized as either passive solar or active solar depending on how they capture and distribute solar energy or convert it into solar power.
Active solar techniques include the use of photovoltaic systems, concentrated solar power, and solar water heating to harness the energy.
Passive solar techniques include orienting a building to the Sun, selecting materials with favorable thermal mass or light-dispersing properties, and designing spaces that naturally circulate air.
In 2011, the International Energy Agency said that "the development of affordable, inexhaustible and clean solar energy technologies will have huge longer-term benefits.
It will increase countries' energy security throu

  1%|          | 1/100 [04:47<7:54:17, 287.45s/it]

Since bare, leafless trees
###########################################################################
George Washington (February 22, 1732 – December 14, 1799 ) was an American military officer, statesman and Founding Father who served as the first president of the United States from 1789 to 1797.
Appointed by the Second Continental Congress as commander of the Continental Army in June 1775, Washington led Patriot forces to victory in the American Revolutionary War and then served as president of the Constitutional Convention in 1787, which drafted and ratified the Constitution of the United States and established the American federal government.
Washington has thus been called the "Father of his Country".
Washington's first public office, from 1749 to 1750, was as surveyor of Culpeper County in the Colony of Virginia.
He subsequently received military training and was assigned command of the Virginia Regiment during the French and Indian War.
He was later elected to the Virginia Hous

  2%|▏         | 2/100 [09:45<7:59:44, 293.72s/it]

...

For example, a writer who is paid to promote a particular product or service may write articles that are biased in favor of that product and service.
###########################################################################
Digital marketing is the component of marketing that uses the Internet and online-based digital technologies such as desktop computers, mobile phones and other digital media and platforms to promote products and services.
Its development during the 1990s and 2000s changed the way brands and businesses use technology for marketing.
As digital platforms became increasingly incorporated into marketing plans and everyday life and as people increasingly used digital devices instead of visiting physical shops, digital marketing campaigns have become prevalent, employing combinations of search engine optimization (SEO), search engine marketing (SEM), content marketing, influencer marketing, content automation, campaign marketing, data-driven marketing, e-commerce marketi

 99%|█████████▉| 99/100 [8:16:37<04:29, 269.44s/it]

Building brand awareness may involve such methods/tools as:=== Search engine optimization (SEO) === Search engine marketing (SEM )
###########################################################################
The United States of America is a federal republic consisting of 50 states, a federal district (Washington, D.C., the capital city of the United States), five major territories, and various minor islands.
Both the states and the United States as a whole are each sovereign jurisdictions.
The Tenth Amendment to the United States Constitution allows states to exercise all powers of government not delegated to the federal government.
Each state has its own constitution and government, and all states and their residents are represented in the federal Congress, a bicameral legislature consisting of the Senate and the House of Representatives.
Each state is represented by two senators, while representatives are distributed among the states in proportion to the most recent constitutionally 

100%|██████████| 100/100 [8:18:39<00:00, 299.20s/it]

== External links == State Resource Guides, from the Library of CongressState and Territorial Governments on USA.gov
###########################################################################


In [8]:
import json
output_file_path = 'distil_Bart.txt'

# Convert the dictionary to JSON format
json_data = json.dumps(final, indent=2)

# Write the JSON data to a text file
with open(output_file_path, 'w') as file:
    file.write(json_data)

print(f'Dictionary has been exported to {output_file_path}')

Dictionary has been exported to distil_Bart.txt
